# Spesifikasi

- V<sub>in</sub> = 48V
- V<sub>out</sub> = 12V
- P<sub>o</sub> = 100W
- &#916;V<sub>o</sub> &#8804; 1%
- &#916;I<sub>L</sub> &#8804; 10%
- f<sub>sw</sub> = 20e3
- Mosfet: IRFB4310PbF, Infineon

# Plecs

In [29]:
import xmlrpc.client as xml
import os
import numpy as np
import itertools
import csv
import pandas as pd


In [30]:
model = 'buck_new'
file_type = '.plecs'
V_in = 48
V_out = 12
f_sw = 20e3

In [31]:
plecs = xml.Server("http://localhost:1080/RPC2").plecs

In [32]:
plecs.load(r"E:\ai-power-converter-1\new\buck_new.plecs")

0

In [39]:
plecs.get(model+'/FETD1')

{'CommentStatus': 'Active',
 'Name': 'FETD1',
 'Ron': '5.6e-3',
 'Rth': '0',
 'T_init': '0',
 'Type': 'MosfetWithDiode',
 's_init': '0',
 'thermal': ''}

In [41]:
plecs.get(model+'/FETD2')

{'CommentStatus': 'Active',
 'Name': 'FETD2',
 'Ron': '5.6e-3',
 'Rth': '0',
 'T_init': '0',
 'Type': 'MosfetWithDiode',
 's_init': '0',
 'thermal': ''}

In [40]:
plecs.get(model+'/L1')

{'CommentStatus': 'Active',
 'L': '0.0007',
 'Name': 'L1',
 'Type': 'Inductor',
 'i_init': '0'}

In [42]:
plecs.get(model+'/DCR')

{'CommentStatus': 'Active', 'Name': 'DCR', 'R': '1e-3', 'Type': 'Resistor'}

In [37]:
plecs.get(model+'/ESR')

{'CommentStatus': 'Active',
 'Name': 'ESR',
 'R': '0.025681987128706025',
 'Type': 'Resistor'}

In [34]:
plecs.get(model+'/C')

{'C': '6e-05',
 'CommentStatus': 'Active',
 'Name': 'C',
 'Type': 'Capacitor',
 'v_init': '0'}

In [44]:
plecs.get(model+'/Load')

{'CommentStatus': 'Active', 'Name': 'Load', 'R': '1', 'Type': 'Resistor'}

In [45]:
plecs.get(model+'/Symmetrical PWM')

{'CommentStatus': 'Active',
 'Name': 'Symmetrical PWM',
 'Type': 'Subsystem',
 'carrier_limits': '[0 1]',
 'carrier_phaseshift': '0',
 'fc': '30000.0',
 'output_values': '[-1 1]',
 'sampling': 'Natural (carrier starts at center)'}

In [46]:
plecs.get(model+'/Duty Cycle')

{'CommentStatus': 'Active',
 'DataType': '10',
 'Name': 'Duty Cycle',
 'Type': 'Constant',
 'Value': '0.25'}

In [38]:
plecs.get(model+'/Deadtime')

{'CommentStatus': 'Active',
 'Configuration': 'on',
 'Name': 'Deadtime',
 'Type': 'ConfigurableSubsystem',
 'td': '876e-9'}

In [12]:
V_in = 48
V_out = 12

#Parameters
R_ON_H = 5.6e-3  # 5.6 milliohms
R_ON_L = 5.6e-3  # 5.6 milliohms

V_D = 0.7  # Diode forward voltage in Volts

C_OSS_H = 540e-12  # 540 pF in Farads

Q_g_H = Q_g_L = 170e-9  # 170 nC in Coulombs
V_GS = 10    

inductor_lookup_table = 'data.csv'  # Path to your CSV file
data_inductor = pd.read_csv(inductor_lookup_table)
d = 0.1  # diameter wire (mm)
rho = 1.68e-5  # resistivitas (Ohm.mm)

tan_delta = 0.14

L_range = (30e-6, 2000e-6)  # Range for L
C_range = (30e-6, 1000e-6)   # Range for C
t_d_range = (175.2e-9, 262.8e-9) # Range for dead time
fsw_range = (20e3, 200e3)    # Range for fsw
num_values = 2          # Number of values for each parameter

L_values = np.linspace(L_range[0], L_range[1], num=num_values)
C_values = np.linspace(C_range[0], C_range[1], num=num_values)
t_d_values = np.linspace(t_d_range[0], t_d_range[1], num=num_values)


# Round the values to the desired number of decimal places
L_values = np.around(L_values, decimals=6)
C_values = np.around(C_values, decimals=6)
t_d_values = np.around(t_d_values, decimals=10)

fsw_values = np.linspace(fsw_range[0], fsw_range[1], num=num_values, dtype=int)

# Initialize arrays to store results
average_currents = []
delta_currents = []
delta_currents_percentage = []

# Open a CSV file for writing
csv_file_path = 'simulation_results.csv'

# Define the header for the CSV file
csv_header = ['No','L', 'C','fsw', 't_d', 'average_current', 'delta_current', 'average_voltage', 'delta_voltage', 'DCR', 'ESR', 'P_ON_H', 'P_ON_L', 'P_COSS', 'P_D', 'P_G', 'P_L_DCR', 'P_CAP_ESR', 'P_total']

# Initialize a list to store the data for each simulation
csv_data = []

# Print the chosen values
print("Chosen L values:", L_values)
print("Chosen C values:", C_values)
print("Chosen dead time values:", t_d_values)
print("Chosen fsw values:", fsw_values)

# Generate all combinations of L, C, and fsw
combinations = list(itertools.product(L_values, C_values, fsw_values, t_d_values))
print(combinations)
# Display the chosen values and simulate
print("\nChosen values and simulation results:")

Chosen L values: [3.e-05 2.e-03]
Chosen C values: [3.e-05 2.e-04]
Chosen dead time values: [1.78e-07 1.00e-06]
Chosen fsw values: [ 20000 200000]
[(3e-05, 3e-05, 20000, 1.78e-07), (3e-05, 3e-05, 20000, 1e-06), (3e-05, 3e-05, 200000, 1.78e-07), (3e-05, 3e-05, 200000, 1e-06), (3e-05, 0.0002, 20000, 1.78e-07), (3e-05, 0.0002, 20000, 1e-06), (3e-05, 0.0002, 200000, 1.78e-07), (3e-05, 0.0002, 200000, 1e-06), (0.002, 3e-05, 20000, 1.78e-07), (0.002, 3e-05, 20000, 1e-06), (0.002, 3e-05, 200000, 1.78e-07), (0.002, 3e-05, 200000, 1e-06), (0.002, 0.0002, 20000, 1.78e-07), (0.002, 0.0002, 20000, 1e-06), (0.002, 0.0002, 200000, 1.78e-07), (0.002, 0.0002, 200000, 1e-06)]

Chosen values and simulation results:


# Functions

## Write to CSV

In [13]:
def write_to_csv(file_path, header, data):
    with open(file_path, 'w', newline='') as csv_file:
        csv_writer = csv.writer(csv_file)
        csv_writer.writerow(header)
        csv_writer.writerows(data)

## DCR

$$
DCR = N \times (OD - ID + 2H) \times r \quad [\Omega]
$$

$$
r = \frac{\rho \cdot l}{A} \quad [\Omega]
$$

Panjang wire:
$$
l = N \times \left( \frac{OD + ID}{2} \times \pi \right) 
$$


## Ripples

### Current
- Average
- &Delta;I<sub>L</sub>%

In [14]:
def calculate_current(times, current, start_time, end_time):
    # Convert 'times' and 'current' to NumPy arrays
    times = np.array(times)
    current = np.array(current)

    # Find the indices corresponding to the time range
    start_index = np.argmax(times >= start_time)
    end_index = np.argmax(times >= end_time)
    
    # Ensure the end_index is correctly set to include the end_time value
    if end_index == 0 and times[end_index] < end_time:
        end_index = len(times)

    # Extract the current values in the specified time range
    current_range = current[start_index:end_index]

    # Calculate the average current value
    average_current = np.mean(current_range)

    # Calculate the difference between the highest and lowest current values
    delta_current = np.max(current_range) - np.min(current_range)

    # Calculate the percentage difference
    delta_current_percentage = (delta_current / average_current) * 100

    return average_current, delta_current, delta_current_percentage


### &Delta;V<sub>O</sub>%

In [15]:
def calculate_voltage(times, voltage, start_time, end_time):
    
    times = np.array(times)
    voltage = np.array(voltage)

    # Find the indices corresponding to the time range
    start_index = np.argmax(times >= start_time)
    end_index = np.argmax(times > end_time)

    # Ensure the end_index is correctly set to include the end_time value
    if end_index == 0 and times[end_index] < end_time:
        end_index = len(times)

    # Extract the voltage values in the specified time range
    voltage_range = voltage[start_index:end_index]

    # Calculate the average voltage value
    average_voltage = np.mean(voltage_range)

    # Calculate the difference between the highest and lowest voltage values
    delta_voltage = np.max(voltage_range) - np.min(voltage_range)
    print('Voltage:')
    print("\nmax:", np.max(voltage_range))
    print("\nmin:", np.min(voltage_range))
    print("\ndelta:",delta_voltage)
    print("\naverage:",average_voltage)

    delta_voltage_percentage = (delta_voltage/average_voltage) * 100

    return average_voltage, delta_voltage, delta_voltage_percentage


## 1. Conduction Losses    
$$
P_{ON-H} = I_{out}^2 \times R_{ON-H} \times \frac{V_{out}}{V_{IN}} \quad \text{[\(W\)]}
$$

$$
P_{ON-L} = I_{OUT}^2 \times R_{ON-L} \times \left( 1 - \frac{V_{OUT}}{V_{IN}} \right) \quad \text{[\(W\)]}
$$

$$
R_{ON-H}=R_{ON-L}=5.6m\Omega
$$

In [ ]:
def calculate_power(I_out, V_out, V_IN, R_ON_H, R_ON_L):
    # Calculate P_ON-H
    P_ON_H = I_out**2 * R_ON_H * (V_out / V_IN)
    
    # Calculate P_ON-L
    P_ON_L = I_out**2 * R_ON_L * (1 - (V_out / V_IN))
    
    return P_ON_H, P_ON_L

## 2. Switching Loss (bisa diskip)

$$
P_{SW-H} = \frac{1}{2} \times V_{in} \times I_{out} \times (t_{r-H} + t_{f-H}) \times f_{sw} \quad \text{[W]}
$$
$$
P_{SW-L} = \frac{1}{2} \times V_{D} \times I_{OUT} \times (t_{r-L} + t_{f-L}) \times f_{SW} \quad \text{[W]}
$$
$$
t_{r-H} = t_{r-L} = 110\text{ns} 
$$
$$
t_{f-H}  = t_{f-L} = 78\text{ns} 
$$
$$
V_D  = 0.7\text{V}
$$


In [16]:
def calculate_switching_loss(V_in, I_out, f_sw, t_r_H, t_r_L, t_f_H, t_f_L, V_D):
    # Calculate P_SW-H
    P_SW_H = 0.5 * V_in * I_out * (t_r_H + t_f_H) * f_sw

    # Calculate P_SW-L
    P_SW_L = 0.5 * V_D * I_out * (t_r_L + t_f_L) * f_sw

    return P_SW_H, P_SW_L

## 3. Output Capacitance Loss
$$
P_{\text{COSS}} = \frac{1}{2} \times C_{\text{OSS-H}} \times V_{\text{IN}}^2 \times f_{\text{SW}} \quad [W]
$$

$$
C_{OSS-H}=540pF

In [ ]:
def calculate_output_capacitance_loss(V_IN, f_sw, C_OSS_H):
    # Calculate P_COSS
    P_COSS = 0.5 * C_OSS_H * V_IN**2 * f_sw

    return P_COSS

## 4. Dead Time Loss
$$
P_{D} = V_{D} \times I_{\text{OUT}} \times t_d \times f_{\text{SW}} \quad [W]
$$
$$
V_D = 0.7\text{V}
$$

In [ ]:
def calculate_dead_time_loss(V_D, I_out, f_sw, t_d):

    # Calculate P_D
    P_D = V_D * I_out * t_d * f_sw

    return P_D


## 5. Gate Charge Loss

$$
P_{G} = (Q_{g-H} + Q_{g-L}) \times V_{gs} \times f_{\text{SW}} \quad [W]
$$
$$
Q_{g-H} = Q_{g-L} = 170nC \\
$$
$$
V_{GS} = 10\text{V}
$$

In [ ]:
def calculate_gate_charge_loss(f_sw, Q_g_H, Q_g_L, V_GS):

    # Calculate P_G
    P_G = (Q_g_H + Q_g_L) * V_GS * f_sw

    return P_G

## 6. Inductor Conduction Loss
$$
P_{L(DCR)} = I_{\text{OUT}}^2 \times \text{DCR} \quad [W]
$$

In [ ]:
def calculate_dcr(input_L, data, d, rho):
    # Menghitung luas penampang kawat
    A = np.pi * (d / 2) ** 2  # luas wire (mm^2)

    # Menentukan baris yang paling mendekati input_L
    data['L_difference'] = abs(data['L(uH)'] - input_L)
    nearest_L_index = data['L_difference'].idxmin()
    selected_row = data.loc[nearest_L_index]

    N = selected_row['N']
    OD = selected_row['OD(mm)']
    ID = selected_row['ID(mm)']
    H = selected_row['Ht(mm)']

    # Menghitung panjang kawat
    l = N * (OD + ID) * np.pi * 0.5

    # Menghitung resistansi kawat
    r = rho * l / A

    # Menghitung DCR
    DCR = N * (OD - ID + 2 * H) * r

    # Menampilkan hasil
    return DCR

def calculate_inductor_conduction_loss(data, d, rho, input_L, I_out):
    # Calculate DCR using the provided function
    DCR = calculate_dcr(data, d, rho, input_L)

    # Calculate P_L(DCR)
    P_L_DCR = I_out**2 * DCR

    return P_L_DCR

## 7. Capacitor Loss
$$
P_{\text{CAP(ESR)}} = I_{\text{CAP(RMS)}}^2 \times \text{ESR} \quad [W]
$$

$$
\text{ESR} = \frac{\tan \delta}{2 \pi f_s C}
$$

$$
tan\delta = 0.14
$$

In [ ]:
def calculate_I_CAP_RMS(delta_IL):
    # Calculate I_CAP_RMS
    I_CAP_RMS = delta_IL / (2 * np.sqrt(3))
    return I_CAP_RMS

def calculate_esr(tan_delta, f_s, C):
    # Calculate ESR
    ESR = tan_delta / (2 * np.pi * f_s * C)
    return ESR

def calculate_capacitor_loss(delta_IL, tan_delta, f_s, C):
    # Calculate I_CAP_RMS
    I_CAP_RMS = calculate_I_CAP_RMS(delta_IL)
    
    # Calculate ESR
    ESR = calculate_esr(tan_delta, f_s, C)
    
    # Calculate P_CAP(ESR)
    P_CAP_ESR = I_CAP_RMS**2 * ESR
    
    return I_CAP_RMS, ESR, P_CAP_ESR

## Total Losses

$$
P = P_{ON-H} + P_{ON-L} + P_{SW-H} + P_{SW-L} + P_{COSS} + P_{D} + P_{G} + P_{L(DCR)} + P_{CAP(ESR)} \quad [W]
$$

In [ ]:
def calculate_total_losses(P_ON_H, P_ON_L, P_COSS, P_D, P_G, P_L_DCR, P_CAP_ESR):
    P_total = P_ON_H + P_ON_L  + P_COSS + P_D + P_G + P_L_DCR + P_CAP_ESR
    return P_total

# Single

In [31]:
val_test =[3e-05,3e-05,20000, 0]

L = val_test[0]
C = val_test[1]
fsw = val_test[2]
t_dt = val_test[3]

# Set Plecs parameters and simulate
plecs.set(model+'/L1', 'L', str(L))
plecs.set(model+'/C', 'C', str(C))
plecs.set(model+'/Symmetrical PWM1', 'fc', str(fsw))
plecs.set(model+'/RL', 'R', str(R_L))
plecs.set(model+'/ESR', 'R', str(ESR))
plecs.set(model+'/ESL', 'L', str(ESL))
plecs.set(model+'/Deadtime', 'td', str(t_dt))

times = plecs.simulate(model)['Time']
current = plecs.simulate(model)['Values'][0]
voltage = plecs.simulate(model)['Values'][1]

# Calculate average and delta current using the function
average_current, delta_current, delta_current_percentage = calculate_current(times, current, start_time=0.004, end_time=0.005)
average_voltage, delta_voltage, delta_voltage_percentage = calculate_voltage(times, voltage, start_time=0.004, end_time=0.005)
Pl_s1, Pl_s2 = calculate_switching_losses(average_current, V_out, V_in, R_on, t_rise, t_fall, fsw)
Pl_C = calculate_capacitor_losses(average_current, V_in, V_out, fsw, L, ESR)
Pl_L_Cu = calculate_inductor_copper_loss(average_current, delta_current, R_L)
# P_D = calculate_dead_time_loss(average_current, fsw, t_dt)


print(f"Average Current between 0.004 and 0.005 seconds: {average_current} A")
print(f"Difference between highest and lowest current values between 0.004 and 0.005 seconds: {delta_current} A")
print(f"Difference between highest and lowest current values between 0.004 and 0.005 seconds: {delta_current_percentage} %")
print(f"Difference between highest and lowest voltage values between 0.004 and 0.005 seconds: {delta_voltage} V")
print(f"Difference between highest and lowest voltage values between 0.004 and 0.005 seconds: {delta_voltage_percentage} %")
print(f"High Side Switching Losses: {Pl_s1} W")
print(f"Low Side Switching  Losses: {Pl_s2} W")
print(f"Capacitor Losses: {Pl_C} W")
print(f"Inductor Copper Losses: {Pl_L_Cu} W")
# print(f"Dead time Losses: {P_D} W")
# print(f"Total Losses: {Pl_s1 + Pl_s2 + Pl_C + Pl_L_Cu + P_D} W")

Voltage:

max: 12.675008038889622

min: 10.055806220980628

delta: 2.619201817908994

average: 11.548067497351191
Average Current between 0.004 and 0.005 seconds: 3.2556026550071735 A
Difference between highest and lowest current values between 0.004 and 0.005 seconds: 14.944351228442894 A
Difference between highest and lowest current values between 0.004 and 0.005 seconds: 459.0348642657058 %
Difference between highest and lowest voltage values between 0.004 and 0.005 seconds: 2.619201817908994 V
Difference between highest and lowest voltage values between 0.004 and 0.005 seconds: 22.680866894047572 %
High Side Switching Losses: 2.5727074420928684 W
Low Side Switching  Losses: 2.5727074420928684 W
Capacitor Losses: 1.5819122871435367 W
Inductor Copper Losses: 2.3244217067618997 W


# Loop

In [21]:
# Loop through the combinations and simulate
csv_data = []
for simulation_num, (L, C, fsw, t_dt) in enumerate(combinations, start=1):
    # Rules for picking R_L based on L
    R_L = next((rl for border, rl in border_values if L <= border), 0.0)

    # Calculate ESR and ESL based on the updated C
    ESR = tan_delta / (2 * np.pi * fsw * C)
    ESL = K_esl / C

    # Set Plecs parameters and simulate
    plecs.set(model+'/L1', 'L', str(L))
    plecs.set(model+'/C', 'C', str(C))
    plecs.set(model+'/Symmetrical PWM1', 'fc', str(fsw))
    plecs.set(model+'/RL', 'R', str(R_L))
    plecs.set(model+'/ESR', 'R', str(ESR))
    plecs.set(model+'/ESL', 'L', str(ESL))
    plecs.set(model+'/Deadtime', 'td', str(t_dt))

    times = plecs.simulate(model)['Time']
    current = plecs.simulate(model)['Values'][0]
    voltage = plecs.simulate(model)['Values'][1]

    # Calculate average and delta current using the function
    average_current, delta_current, delta_current_percentage = calculate_current(times, current, start_time=0.004, end_time=0.005)
    average_voltage, delta_voltage, delta_voltage_percentage = calculate_voltage(times, voltage, start_time=0.004, end_time=0.005)
    Pl_s1, Pl_s2 = calculate_switching_losses(average_current, V_out, V_in, R_on, t_rise, t_fall, fsw)
    Pl_C = calculate_capacitor_losses(average_current, V_in, V_out, fsw, L, ESR)
    Pl_L_Cu = calculate_inductor_copper_loss(average_current, delta_current, R_L)
    P_D = calculate_dead_time_loss(average_current, fsw, t_dt)

    # Append the data for this simulation to the list
    csv_data.append([simulation_num,L, C, fsw, t_dt, delta_current_percentage, delta_voltage_percentage, Pl_s1, Pl_s2, Pl_C, Pl_L_Cu, P_D])

    print(f'\nSimulation {simulation_num} for L={L:.6f}, C={C:.6f}, fsw={fsw:.2f}, RL={R_L:.6f}, ESR={ESR:.6f}, ESL={ESL:.6e}')
    print(f"\nSimulation {simulation_num} results:")
    print(f"Average Current between 0.004 and 0.005 seconds: {average_current} A")
    print(f"Difference between highest and lowest current values between 0.004 and 0.005 seconds: {delta_current} A")
    print(f"Difference between highest and lowest current values between 0.004 and 0.005 seconds: {delta_current_percentage} %")
    print(f"Difference between highest and lowest voltage values between 0.004 and 0.005 seconds: {delta_voltage} A")
    print(f"Difference between highest and lowest voltage values between 0.004 and 0.005 seconds: {delta_voltage_percentage} %")
    print(f"High Side Switching Losses: {Pl_s1} W")
    print(f"Low Side Switching  Losses: {Pl_s2} W")
    print(f"Capacitor Losses: {Pl_C} W")
    print(f"Inductor Copper Losses: {Pl_L_Cu} W")
    print(f"Dead time Losses: {P_D} W")
    print(f"Total Losses: {Pl_s1 + Pl_s2 + Pl_C + Pl_L_Cu + P_D} W")

Voltage:

max: 12.679319769676034

min: 10.055395242914386

delta: 2.6239245267616482

average: 11.754836343978347

Simulation 1 for L=0.000030, C=0.000030, fsw=20000.00, RL=0.079576, ESR=0.037136, ESL=1.463000e-06

Simulation 1 results:
Average Current between 0.004 and 0.005 seconds: 2.369076212400351 A
Difference between highest and lowest current values between 0.004 and 0.005 seconds: 14.942629474599412 A
Difference between highest and lowest current values between 0.004 and 0.005 seconds: 630.7365460167921 %
Difference between highest and lowest voltage values between 0.004 and 0.005 seconds: 2.6239245267616482 A
Difference between highest and lowest voltage values between 0.004 and 0.005 seconds: 22.322084714567776 %
High Side Switching Losses: 1.8721387860872531 W
Low Side Switching  Losses: 1.8721387860872531 W
Capacitor Losses: 1.1652647096399509 W
Inductor Copper Losses: 1.927280592538472 W
Dead time Losses: 0.00843391131614525 W
Total Losses: 6.845256785669075 W
Voltage:

m

In [22]:
# Use the function to write data to the CSV file
write_to_csv(csv_file_path, csv_header, csv_data)